# Selenium
透過 WebDriver 來驅動瀏覽器，以便進行網頁元素互動的工具/套件，可以想像透過程式來決定如何操作瀏覽器的過程。

In [ ]:
!pip install selenium webdriver-manager fake-useragent 

# 頁面走訪
- driver.get('目標網址')


# 定位元素
- 使用 webdriver 物件，例如 driver
  - driver.find_element(): 取得單一元素
  - driver.find_elements(): 取得元素集合 (回傳 list)
  - driver.find_element_by_*: 在近期的 WebChrome 版本中棄用


# 元素定位的策略/方式 (By)
- 例如 driver.find_element(By.CSS_SELECTOR, '選擇器字串')
  - By.ID = "id"
  - **By.CSS_SELECTOR = "css selector"**
  - By.XPATH = "xpath"
  - By.LINK_TEXT = "link text"
  - By.PARTIAL_LINK_TEXT = "partial link text"
  - By.NAME = "name"
  - By.TAG_NAME = "tag name"
  - By.CLASS_NAME = "class name"


# 期待狀況/條件（expected_conditions）
- 例如 expected_conditions.presence_of_element_located(By.CSS_SELECTOR, '選擇器字串')
  - **presence_of_element_located**
    - 判斷某個元素是否被加到了dom樹裡，並不代表該元素一定可見
  - visibility_of_element_located
    - 判斷元素是否可見. 可見代表元素非隱藏，並且元素的寬和高都不等於0


# 等待
- **強制等待 (sleep)**
  - 通常泛指 sleep() 函式
- 隱性等待 (implicitly_wait)
  - 設置了一個最長等待時間，如果在規定時間內網頁加載完成，或是能夠取得指定的元素（透過 find_element*），則執行下一步，否則一直等到時間截止，然後拋出例外。
- **顯性等待 (WebDriverWait)**
  - 配合 until() 和 until_not() 方法，就能夠根據判斷條件而進行靈活地等待了。它主要的意思就是：如果條件成立了，則執行下一步，否則繼續等待，直到超過設置的最長時間，直到拋出TimeoutException。

```python
'''
WebDriverWait(WebDriver, 等待秒數).until(
    期待.條件(
        (元素定位策略, '選擇器字串')
    )
)
'''
try:
    WebDriverWait(driver, 10).until(
        expected_conditions.presence_of_element_located(
            (By.CSS_SELECTOR, '選擇器字串')
        )
    )
    .
    .
    確認元素存在於 DOM 之中，程式碼便會繼續往下執行...
    .
    .
except TimeoutException:
    print("等待逾時的處置")
    
```


# 元素互動
- 點擊 (click)
  - 例如 driver.find_element(By.CSS_SELECTOR, '選擇器字串')


# 下拉式選單
- 例如 sel_element = Select(driver.find_element(By.CSS_SELECTOR, '選擇器字串'))
  - 透過 option 的內文來選擇
    - sel_element.select_by_visible_text('民國 100 年')
  - 透過 option 的 value 屬性所設定值來選擇
    - sel_element.select_by_value('2')
  - 透過 option 的順序索引 (從 0 開始，類似陣列的索引概念) 來選擇
    - sel_element.select_by_index(8)


# 擷圖
- driver.save_screenshot('/path/圖片存放路徑.png')

In [ ]:
'''
臺灣證券交易所
外資及陸資買賣超彙總表
https://www.twse.com.tw/zh/page/trading/fund/TWT38U.html

目標:
整合下拉式選單與元素的定位與操控，來下載交易資料，並擷圖
'''

'''
匯入套件
'''
# 操作 browser 的 驅動程式
from selenium import webdriver

# 負責開啟和關閉 Chrome 的套件
from selenium.webdriver.chrome.service import Service

# 自動下載 Chrome Driver 的套件
from webdriver_manager.chrome import ChromeDriverManager

# 例外處理的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待、了解某個元素的狀態，通常與 exptected_conditions 和 By 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 取得系統時間的工具
from datetime import datetime

# 強制停止/強制等待 (程式執行期間休息一下)
from time import sleep

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent()

'''
[1] Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
[2] Selenium with Python
https://selenium-python.readthedocs.io/
[3] selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
[4] How to select a drop-down menu value with Selenium using Python?
參考網址：https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
'''

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         # 最大化視窗
my_options.add_argument("--incognito")               # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")   # 取消通知
my_options.add_argument(f'--user-agent={ua.random}') # (Optional)加入 User-Agent


# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'files')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

    
#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})


# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)


# 走訪網址
url = 'https://www.twse.com.tw/zh/page/trading/fund/TWT38U.html'

In [ ]:
# 走訪頁面
def visit():
    driver.get(url);

# 選取下拉式選單的項目
def setDropDownMenu():
    try:
        # 強制等待
        sleep(1)

        # 選擇 select[name="yy"] 元素，並依 option 的 innerText 來進行選取
        yy = Select(driver.find_element(By.CSS_SELECTOR, 'div#d1 > select[name="yy"]'))
        yy.select_by_visible_text('民國 100 年')

        # 強制等待
        sleep(1)

        # 選擇 select[name="mm"] 元素，並依 option 的 value 來進行選取
        mm = Select(driver.find_element(By.CSS_SELECTOR, 'div#d1 > select[name="mm"]'))
        mm.select_by_value('2')

        # 強制等待
        sleep(1)

        # 選擇 select[name="dd"] 元素，並依 option 的 index 來進行選取
        dd = Select(driver.find_element(By.CSS_SELECTOR, 'div#d1 > select[name="dd"]'))
        dd.select_by_index(8)

        # 強制等待
        sleep(1)

        # 按下查詢
        driver.find_element(
            By.CSS_SELECTOR, 
            'a.button.search'
        ).click()

        # 強制等待
        sleep(2)
    except Exception as err:
        print(str(err))
        driver.quit()
    
# 下載檔案
def download():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, "div.tools > a.csv") 
            )
        )
        
        # 下載
        driver.find_element(
            By.CSS_SELECTOR, 
            "div.tools > a.csv"
        ).click()
        
        # 強制等待
        sleep(2)
        
        # 找出現在時間 (將 timestamp 轉成 年月日時分秒 的字串格式)
        strDateTime = datetime.today().strftime("%Y%m%d%H%M%S")
        
        # 擷圖
        driver.save_screenshot(f"{folderPath}/{strDateTime}.png");
        
        # 強制等待
        sleep(2)
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        driver.quit()

# 關閉瀏覽器
def close():
    driver.quit()

In [ ]:
# 主程式
if __name__ == '__main__':
    visit()
    setDropDownMenu()
    download()

In [ ]:
# 關閉瀏覽器
close()